# PLCTestbench

In [ ]:
from plctestbench.settings import *
from plctestbench.plc_testbench import PLCTestbench
from plctestbench.loss_simulator import BinomialPLS, GilbertElliotPLS
from plctestbench.plc_algorithm import ZerosPLC, LastPacketPLC, LowCostPLC, ExternalPLC, DeepLearningPLC
from plctestbench.output_analyser import MSECalculator, PEAQCalculator, SpectralEnergyCalculator

testbench_settings = {
    'root_folder': '../original_tracks',
    'db_ip': '192.168.122.1',
    'db_username': 'myUserAdmin'
}
user = {
    'email': 'luca.vignati@vignati.net',
    'first_name': 'Luca',
    'last_name': 'Vignati',
    'locale': 'it_IT',
    'image_link': ''
}

packet_loss_simulators = [(GilbertElliotPLS, GilbertElliotPLSSettings(seed=234, p=0.001, r=0.05, h=0.5, k=0.99999900)),
                          (BinomialPLS, BinomialPLSSettings(seed=1, per=0.005))]
ecc_algorithms = [(LowCostPLC, LowCostPLCSettings()),
                  (ZerosPLC, ZerosPLCSettings())]
output_analysers = [(MSECalculator, MSECalculatorSettings()),
                    (PEAQCalculator, PEAQCalculatorSettings(peaq_mode="basic"))]

testbench = PLCTestbench(packet_loss_simulators, ecc_algorithms, output_analysers, testbench_settings, user)

In [ ]:
testbench.run()

In [ ]:
%matplotlib inline
testbench.plot(show=False, to_file=True, group=False, original_tracks=True, lost_samples_masks=True, output_analyses=True, peaq_summary=False)